## Check the setup and connect to the database

In [1]:
%run "../01-check_setup.ipynb"

SAP HANA Client for Python: 2.26.25111700
Connected to SAP HANA db version 4.00.000.00.1764669230 (fa/CE2025.28) 
at c5889dd5-e0f6-4930-8408-94d53ca61dbf.hna0.prod-us10.hanacloud.ondemand.com:443 as CODEJAMHANAAI00
Current time on the SAP HANA server: 2025-12-12 20:42:56.418000


In [2]:
myconn.get_tables(schema='NHTSA')

,TABLE_NAME


In [3]:
import pandas as pd

Loading data from https://www.nhtsa.gov/nhtsa-datasets-and-apis#complaints

In [4]:
import_df = pd.read_csv('https://static.nhtsa.gov/odi/ffdd/cmpl/COMPLAINTS_RECEIVED_2020-2024.zip', sep='\t', header=None)
import_df.shape

/tmp/ipykernel_10266/1342734287.py:1: DtypeWarning: Columns (27,28,29,30,33,38,41,44) have mixed types. Specify dtype option on import or set low_memory=False.
  import_df = pd.read_csv('https://static.nhtsa.gov/odi/ffdd/cmpl/COMPLAINTS_RECEIVED_2020-2024.zip', sep='\t', header=None)


(418788, 49)

In [5]:
import_df.rename(columns={
    0: 'CMPLID',
    1: 'ODINO',
    2: 'MFR_NAME',
    3: 'MAKETXT',
    4: 'MODELTXT',
    5: 'YEARTXT',
    6: 'CRASH',
    7: 'FAILDATE',
    8: 'FIRE',
    9: 'INJURED',
    10: 'DEATHS',
    11: 'COMPDESC',
    12: 'CITY',
    13: 'STATE',
    14: 'VIN',
    15: 'DATEA',
    16: 'LDATE',
    17: 'MILES',
    18: 'OCCURENCES',
    19: 'CDESCR',
    20: 'CMPL_TYPE',
    21: 'POLICE_RPT_YN',
    22: 'PURCH_DT',
    23: 'ORIG_OWNER_YN',
    24: 'ANTI_BRAKES_YN',
    25: 'CRUISE_CONT_YN',
    26: 'NUM_CYLS',
    27: 'DRIVE_TRAIN',
    28: 'FUEL_SYS',
    29: 'FUEL_TYPE',
    30: 'TRANS_TYPE',
    31: 'VEH_SPEED',
    32: 'DOT',
    33: 'TIRE_SIZE',
    34: 'LOC_OF_TIRE',
    35: 'TIRE_FAIL_TYPE',
    36: 'ORIG_EQUIP_YN',
    37: 'MANUF_DT',
    38: 'SEAT_TYPE',
    39: 'RESTRAINT_TYPE',
    40: 'DEALER_NAME',
    41: 'DEALER_TEL',
    42: 'DEALER_CITY',
    43: 'DEALER_STATE',
    44: 'DEALER_ZIP',
    45: 'PROD_TYPE',
    46: 'REPAIRED_YN',
    47: 'MEDICAL_ATTN',
    48: 'VEHICLES_TOWED_YN'
}, inplace=True)

In [6]:
# Combine them into a single DataFrame
pd.DataFrame({
    'Original Dtypes': import_df.dtypes,
    'Converted Dtypes': import_df.convert_dtypes().dtypes
})

,Original Dtypes,Converted Dtypes
CMPLID,int64,Int64
ODINO,int64,Int64
MFR_NAME,object,string[python]
MAKETXT,object,string[python]
MODELTXT,object,string[python]
YEARTXT,float64,Int64
CRASH,object,string[python]
FAILDATE,int64,Int64
FIRE,object,string[python]
INJURED,int64,Int64


In [7]:
import_df=import_df.convert_dtypes()

In [8]:
import_df[import_df['CDESCR'].str.len() > 2048][['CMPLID']].assign(CDESCR_LEN=import_df['CDESCR'].str.len())

,CMPLID,CDESCR_LEN
261483,1894904,19513
285331,1918773,9370
299216,1932669,10096
301213,1934684,17330
330785,1964283,2196


In [9]:
pd.set_option('max_colwidth', None) 
display(
    import_df
    .head(1).T
    .style.set_properties(subset=[0], **{'text-align': 'left'})
)

,0
CMPLID,1633421
ODINO,11292384
MFR_NAME,Honda (American Honda Motor Co.)
MAKETXT,HONDA
MODELTXT,ACCORD
YEARTXT,2018
CRASH,N
FAILDATE,20191221
FIRE,N
INJURED,0


In [10]:
carcomplaints_hdf = hana_ml.dataframe.create_dataframe_from_pandas(myconn,
                                           import_df[import_df['CDESCR'].str.len() <= 2048],
                                            table_name="COMPLAINTS",
                                            schema='NHTSA',
                                            force=True)

100%|██████████| 9/9 [00:23<00:00,  2.59s/it]


In [11]:
carcomplaints_hdf.shape

[418758, 49]

In [12]:
myconn.get_tables(schema="NHTSA")

,TABLE_NAME
0,COMPLAINTS


In [13]:
myconn.table("COMPLAINTS", schema='NHTSA').filter("LEFT(LDATE,4) = '2024'").describe().collect().sort_values(by='unique', ascending=True)

,column,count,unique,nulls,mean,std,min,max,median,25_percent_cont,25_percent_disc,50_percent_cont,50_percent_disc,75_percent_cont,75_percent_disc
9,OCCURENCES,0,0,97977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,TIRE_SIZE,0,0,97977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,ORIG_OWNER_YN,96476,2,1501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,POLICE_RPT_YN,97977,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,CRUISE_CONT_YN,96476,2,1501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,FUEL_SYS,47,2,97930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,FIRE,97977,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,ANTI_BRAKES_YN,96476,2,1501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,CMPL_TYPE,97977,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,ORIG_EQUIP_YN,509,2,97468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
